# Interact with decay data in ENDF-6 format (MF=8, MT=457)

First import `sandy` and other packages importing for formatting and postprocessing that are used in this notebook.

In [1]:
import os
import urllib
import yaml

import pandas as pd

In [2]:
%%capture --no-stdout
import sandy

## Read decay data in ENDF-6 format

As always, any nuclear data evaluation can be imported into a `Endf6` instance.

Here we import the JEFF-3.3 radioactive decay data library from the [OECD/NEA website](https://www.oecd-nea.org/dbdata/jeff/jeff33/index.html).

In [3]:
url = "https://www.oecd-nea.org/dbdata/jeff/jeff33/downloads/JEFF33-rdd_all.asc"
with urllib.request.urlopen(url) as f:
    text = f.read().decode('utf-8')

In [4]:
tape = sandy.Endf6.from_text(text)
tape

MAT   MF  MT 
1     1   451     1.000000+0 1.000000+0         -1          0  ...
      8   457     1.000000+0 1.000000+0          0          0  ...
2     1   451     1.001000+3 9.991670-1         -1          0  ...
      8   457     1.001000+3 9.991670-1          0          0  ...
3     1   451     1.002000+3 1.996800+0         -1          0  ...
                                       ...                        
3850  8   457     1.102730+5 2.708020+2          0          0  ...
3851  1   451     1.102730+5 2.708020+2         -1          0  ...
      8   457     1.102730+5 2.708020+2          1          1  ...
3852  1   451     1.112720+5 2.698160+2         -1          0  ...
      8   457     1.112720+5 2.698160+2          0          0  ...
Length: 7731, dtype: object

The ENDF-6 data in `tape` can be parsed and extracted into a hierarchical format (nested `dict`) using method `DecayData.from_endf6`.

In [5]:
rdd = sandy.DecayData.from_endf6(tape)

A better rendering of the `rdd` data content can be obtained with the `yaml` python package.

In [6]:
print(yaml.dump(rdd.data[551340]))

decay_constant: 1.063988758436722e-08
decay_energy:
  alpha: 0.0
  beta: 163850.0
  gamma: 1554660.0
  total: 1718510.0
decay_energy_uncertainties:
  alpha: 0.0
  beta: 403.088
  gamma: 895.993
  total: 982.4883672558165
decay_modes:
  10x0:
    branching_ratio: 0.9999969999999999
    branching_ratio_uncertainty: 1.0e-06
    decay_products:
      561340: 1.0
  20x0:
    branching_ratio: 3.0e-06
    branching_ratio_uncertainty: 1.0e-06
    decay_products:
      541340: 1.0
half_life: 65146100.0
parity: 1.0
spin: 4.0
stable: false



In [7]:
print(yaml.dump(rdd.data[400900]))

decay_constant: 0
decay_energy:
  alpha: 0.0
  beta: 0.0
  gamma: 0.0
  total: 0.0
decay_energy_uncertainties:
  alpha: 0.0
  beta: 0.0
  gamma: 0.0
  total: 0.0
half_life: 0.0
parity: 1.0
spin: 0.0
stable: true



The description of the `rdd.data` structure is explained below, where `zam` and `zap` are 
equal to `Z * 10000 + A * 10 + M` for the parent and daughter isotopes, respectively, with 
`Z`, `A`, and `M` being the charge, nucleons and metastate numbers.

```yaml
    zam :
        half_life : value in s
        decay_constant : value in 1/s
        stable : True/False
        decay_energy :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_energy_uncertainty :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_modes :
            rtyp : 
                decay_products : 
                    zap : yield
                    zap_2 : yield_2
                    ...
                branching_ratio : value
            rtyp_2 : ...
    zam_2 : ...
```

`rtyp` is a string storing a sequence of integers, each of which defining the 
several decay events covered by a given decay mode.
The list of individual decay events is accessible via variable `sandy.decay_modes`.

In [8]:
sandy.decay_modes

{0: 'gamma',
 1: 'beta',
 2: 'e.c.',
 3: 'i.t.',
 4: 'alpha',
 5: 'n',
 6: 's.f.',
 7: 'p'}

## Extract structured information into dataframes

We can extract the decay chains information (decay constant `LAMBDA` and branching ratio `YIELD`) into a `pandas.DataFrame`.

In [9]:
chains = rdd.get_decay_chains()
chains

,PARENT,DAUGHTER,YIELD,LAMBDA
0,10,10,-1.00000e+00,1.12780e-03
1,10,10010,1.00000e+00,1.12780e-03
2,10010,10010,0.00000e+00,0.00000e+00
3,10020,10020,0.00000e+00,0.00000e+00
4,10030,10030,-1.00000e+00,1.78139e-09
...,...,...,...,...
10375,1102731,1082690,1.00000e+00,5.77623e+00
10376,1102731,1102731,-1.00000e+00,5.77623e+00
10377,1112720,20040,1.00000e+00,3.46574e+02
10378,1112720,1092680,1.00000e+00,3.46574e+02


## Convert decay data into HDF5 format

Codes like ALEPH read radioactive decay data from nuclear data files in HDF5 format.
To produce such files we can use `sandy` and method `DecayData.to_hdf5`.

As an example, we can transfer the whole content of the `fy` instance to a HDF5 file `'decay.hdf5'`.

In [10]:
%%capture
h5filename = "decay.hdf5"
libname = "jeff_33"
if os.path.exists(h5filename):
    os.remove(h5filename)
rdd.to_hdf5(h5filename, libname)

Decay data can also be read directly from a HDF5 file using method `DecayData.from_hdf5`.

In [11]:
rdd2 = sandy.DecayData.from_hdf5(h5filename, libname)

In [12]:
chains2 = rdd2.get_decay_chains()
pd.testing.assert_frame_equal(chains, chains2)